In [ ]:
import xarray as xr
import numpy as np
import cartopy
from cartopy import crs as ccrs
import matplotlib 
matplotlib.rcParams["font.size"] = 12
from matplotlib import pyplot as plt
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import sklearn
import sys
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
%matplotlib inline
import matplotlib.path as mpath
import importlib.util

import warnings
import pickle

warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import shap

from sklearn.metrics import confusion_matrix,recall_score,precision_score

import os
import matplotlib as mpl
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
import MM_util_AI
import MM_utilplot
from matplotlib import cm 
spec = importlib.util.spec_from_file_location("MM_dataprepare", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_dataprepare.py")
MM_dataprepare = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_dataprepare
spec.loader.exec_module(MM_dataprepare)

spec = importlib.util.spec_from_file_location("MM_utilblocking", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_utilblocking.py")
MM_utilblocking = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_utilblocking
spec.loader.exec_module(MM_utilblocking)

    
from scipy.stats import wilcoxon
   
            
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("code_filepath"):
            code_path=ln.strip().split('\t')[1]            
        if ln.startswith("DGindex_filepath"):
            DGindex_path=ln.strip().split('\t')[1]  
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1] 
print(dim_path)
print(nondim_path)
print(code_path)
print(DGindex_path)
print(train_path)
print(models_path)
print(fig_path)

In [ ]:
def extract_era5_certain_testset(t):
    precision_list=[]
    recall_list=[]    
    if class_imb_Method=="":
        range_random_seed=np.arange(10)
    elif class_imb_Method=="_oversampling":
        range_random_seed=np.arange(30,40)
    else:
        print("error")
    for random_seed in range_random_seed:
        path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN"+class_imb_Method+\
                "/extreme_%ddaysblocking/%s/"%(Duration,cnnsize)\
                +"random_initCNN_%d/"%random_seed
        precision_direct=np.load(path+"precision_direct.npy") 
        recall_direct=np.load(path+"recall_direct.npy") 
        precision_list.append(precision_direct[t] )
        recall_list.append(recall_direct[t] )
    return np.array(precision_list),np.array(recall_list)

def transfer(t,epoch):
    precision_list=[]
    recall_list=[]
    for random_seed in range(30,40):
        path=path0+ "data_"+data_amount+"_"+cnnsize+"_cnn_"+"regularize"+"_%.3e"%epsilon+"_rs_%d"%random_seed+\
                    "epoch_%d/"%epoch
        precision_transfer=np.load(path+"precision_transfer.npy") 
        recall_transfer=np.load(path+"recall_transfer.npy") 
        precision_list.append(precision_transfer[t] )
        recall_list.append(recall_transfer[t] )
    return np.array(precision_list),np.array(recall_list)
def climatology_pred(test_data_list,Ysparse,climatology_testset):
    climatology_precision=np.zeros((num,10))
    climatology_recall=np.zeros((num,10))
    for i in range(num):
        random_seed_era5=i
        test_ind=test_data_list[i]
        test_labels=Ysparse[test_ind]
        np.random.seed(random_seed_era5)
        climatology_predict=(np.random.uniform(size=(10, test_labels.size)) < climatology_testset[i]) * 1
        for j in range(10):
            TP=np.logical_and(test_labels==1, climatology_predict[j]==1).sum()
            TF=np.logical_and(test_labels==0, climatology_predict[j]==0).sum()
            FP=np.logical_and(test_labels==0, climatology_predict[j]==1).sum()
            FN=np.logical_and(test_labels==1, climatology_predict[j]==0).sum()
            climatology_recall[i,j]=TP/(TP+FN)
            climatology_precision[i,j]=TP/(TP+FP)
    return climatology_precision,climatology_recall

In [ ]:
epsilon=0.0
data_amount='1000.0k'
cnnsize='normal'
Duration=5
# class_imb_Method="_oversampling"
class_imb_Method=""

if Duration==5 and class_imb_Method=="":
    epoch=2 # epoch of fine-tuning
elif Duration==7 and class_imb_Method=="":
    epoch=1
else:
    print("not computed yet")

tname="7"
learning_rate="0.0001"

X=np.load("/scratch/hz1994/blocking/data_era5/"+"test_data_1940-2022.npy")  

if Duration==5:
    Ysparse=np.load("/scratch/hz1994/blocking/data_era5/"+"test_labels_1940-2022_T%d.npy"%Duration)
elif Duration==7 or Duration==9:
    Ysparse=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_%dd/T/test_labels.npy"%Duration)[:,1]

Y=np.zeros((Ysparse.size,2)).astype(bool)
Y[:,1][Ysparse==1]=True  #blocking
Y[:,0][Ysparse==0]=True
seed=10
num=10
num_total=X.shape[0]
ordered_ind=np.arange(num_total)
np.random.seed(seed)
np.random.shuffle(ordered_ind)
shuffled_ind=np.random.shuffle(ordered_ind)
test_data_list=(np.array_split(ordered_ind, num))
train_data_list=[]
for i in range(num):
    train_data_list.append( np.array( [j  for j in ordered_ind if j not in test_data_list[i]]   )  )
    
climatology_testset=[]
for i in range(num):
    # load data
    train_ind=train_data_list[i]
    train_labels=Y[train_ind]
    climatology_testset.append(train_labels[:,1].sum()/train_labels[:,1].size)
    print(Y[test_data_list[i],1].sum())
    
climatology_precision,climatology_recall=climatology_pred(test_data_list,Ysparse,climatology_testset)
climatology_precision=climatology_precision[...,None]
climatology_recall=climatology_recall[...,None]

climatology_precision_t=np.nanmean(climatology_precision,axis=(1,2))
climatology_precision_t_std=np.nanstd(climatology_precision_t)
climatology_precision_t_mean=np.nanmean(climatology_precision_t)

climatology_recall_t=np.nanmean(climatology_recall,axis=(1,2))
climatology_recall_t_std=np.nanstd(climatology_recall_t)
climatology_recall_t_mean=np.nanmean(climatology_recall_t)


path0="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_retrainCNN"+class_imb_Method+"/"+\
                "extreme_%ddaysblocking/"%Duration+\
                "trained_layer_%s/learning_rate_%s/"%(tname, learning_rate)

# extract the data from ERA5 direct training and transfer learning starting from pre-training epoch =2
precision_list=[]
recall_list=[]
era5precision_list=[]
era5recall_list=[]    
for tind,t in enumerate(range(10)):# the 7th training-test set pair
    precision_t,recall_t=transfer(t,epoch)    
    era5precision_t,era5recall_t=extract_era5_certain_testset(t)     
    era5precision_list.append(era5precision_t )
    era5recall_list.append(era5recall_t)
    precision_list.append(precision_t)
    recall_list.append(recall_t)

era5precision_list=np.array(era5precision_list) 
era5recall_list=np.array(era5recall_list) 
precision_list=np.array(precision_list) 
recall_list=np.array(recall_list) 

era5precision=np.nanmean(era5precision_list,axis=1)
era5recall=np.nanmean(era5recall_list,axis=1)
precision=np.nanmean(precision_list,axis=1)
recall=np.nanmean(recall_list,axis=1)

print("path0=",path0)
np.save(path0+"plot/era5precision.npy",era5precision)
np.save(path0+"plot/era5recall.npy",era5recall)
np.save(path0+"plot/precision.npy",precision)
np.save(path0+"plot/recall.npy",recall)
np.save(path0+"plot/climatology_precision_t_mean.npy",climatology_precision_t_mean)
np.save(path0+"plot/climatology_recall_t_mean.npy",climatology_recall_t_mean)